# Project plan

In [ ]:
"https://miro.com/app/board/uXjVKz7_SX8=/?share_link_id=88913006867"

# Lib

In [ ]:
# Для парсинга данных
!pip install telethon nest_asyncio


In [ ]:
# Для обработки данных

!pip install emoji vaderSentiment bertopic num2words pymorphy2
!python -m spacy download ru_core_news_sm


In [ ]:
# Для cуммирования информации и RAG
!pip install transformers
!pip install accelerate -U

In [ ]:
from telethon import TelegramClient, types, functions
from telethon.tl.functions.channels import GetFullChannelRequest
import nest_asyncio

In [ ]:
import emoji
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from num2words import num2words
import pymorphy2
from bertopic import BERTopic

In [ ]:
from transformers import pipeline

In [1]:
import warnings
warnings.filterwarnings('ignore')
random_state = 17

import pandas as pd
import numpy as np

from datetime import datetime
import matplotlib as mlp
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import spacy
import string

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

from scipy.stats import hmean
from scipy.stats import boxcox

In [ ]:
from decouple import config

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

# Code for mining data

In [ ]:
nest_asyncio.apply()
# Личныве данные для аккаунта

api_id = config('API_ID')
api_hash = config('API_HASH')
phone_number = config('PHONE_NUMBER')

In [ ]:
async def registr(phone_number, api_id, api_hash) -> None:
    client = TelegramClient(phone_number, api_id, api_hash)
    await client.start()
    print(await client.get_me())
    return client
    
client = registr(phone_number, api_id, api_hash)

In [ ]:
async def reactions_to_str(client, reactions: types.MessageReactions) -> str:
    if reactions:
        reactions_list = []
        for reaction in reactions.results:
            if isinstance(reaction.reaction, types.ReactionEmoji):
                emoji = reaction.reaction.emoticon
                reactions_list.append(f"{emoji}: {reaction.count}")
            elif isinstance(reaction.reaction, types.ReactionCustomEmoji):
                reactions_list.append(f"Custom emoji: {reaction.count}")
        return ", ".join(reactions_list)
    else:
        return "No reactions"
async def count_reactions(reactions):
    if reactions:
        return sum([r.count for r in reactions.results])
    return 0

async def fetch_messages(channel_username, limit=1000):
    async with client:
        channel = await client.get_entity(channel_username)
        name = channel.title
        full_channel = await client(GetFullChannelRequest(channel))
        messages = []
        async for message in client.iter_messages(channel, limit=limit):
            message_data = {
                'date': message.date,
                'name': name,
                'text': message.text,
                'views': message.views,
                'comments': message.replies.replies if message.replies else 0,
                'forwards': message.forwards if message.forwards else 0,
                'emoji': await reactions_to_str(client, message.reactions) if message.reactions else "No reactions",
                'reactions': await count_reactions(message.reactions),
                'subscribers': full_channel.full_chat.participants_count
            }
            messages.append(message_data)
        return messages

In [ ]:
channels = ['https://t.me/data_science_winners',
            'https://t.me/sharemed',
            'https://t.me/machinelearning_interview',
            'https://t.me/econs',
            'https://t.me/datascienceml_jobs',
            'https://t.me/pravdadirty',
            'https://t.me/betboom_esports',
            ]

def loop_data(channels):
  messages = []
  for channel in channels:
    messages_channel = client.loop.run_until_complete(fetch_messages(channel))
    messages.extend(messages_channel)
  return messages


In [ ]:
messages = loop_data(channels)

In [ ]:
df = pd.DataFrame(messages)
df.head()

In [ ]:
# удалим пустые строки где нет текста (такие появляются из-за редактирования постов или фотографий, которые считываются как посты)
df_clean = df[df['text'].notna()]

# приводим дату к обычному виду
df_clean['date'] = df_clean['date'].dt.tz_localize(None)

# индексы сбрасываем
df_clean = df_clean.reset_index(drop=True)
df_clean

In [ ]:
df_clean.to_csv('telegram_posts.csv', index=False, sep = ',', encoding = 'utf-8')

# data


In [ ]:
"""
Данные взял из 7 каналов, пока тестирую возможности на данном наборе данных
(расценивается как сэмпл) для понимания что вообще нужно делать.
"""

In [ ]:
data = pd.read_csv('telegram_posts.csv', sep = ',', parse_dates=['date'], encoding = 'utf-8')
data.head()

**data** - Дата поста

**name** - Название канала

**text** - Текст поста

**views** - Кол-во просмотров

**comments** - Кол-во комментариев

**forwards** - Кол-во раз, которые пересылали пост

**emoji** - Список реакций в формате "emoji: кол-во"

**reactions** - Кол-во реакций

**subscribers** - Кол-во подписчиков

In [ ]:
# После загрузки данных проверим повторно наличие пропусков
# Уберем пропуски в тексте, которые остались после обработки
data.info()
data = data.dropna(subset='text')
print('\n')
data.info()

In [ ]:
# Остались нули только в столбце emoji, заполним их нулями
# Потому что в реакциях стоит 0, просто неправильная обработка
data.fillna(0, inplace=True)
data.info()

In [ ]:
# Переведем в целочисленные значения для удобства
data['views'] = data['views'].astype('int')
data.info()

In [ ]:
# Отобразим конверсию просмотров в три оснвных показателя:
data['comms_views'] = data['comments'] / data['views']
data['forwards_views'] = data['forwards'] / data['views']
data['reactions_views'] = data['reactions'] / data['views']
data.head()

In [ ]:
data.describe()

# analys

In [ ]:
sns.histplot(data['comms_views'], bins=5, kde=True)
plt.xlabel('Значения')
plt.ylabel('Частота')
plt.title('Гистограмма распределения значений с KDE')
plt.show()

sns.kdeplot(data['comms_views'], shade=True)
plt.xlabel('Значения')
plt.title('Диаграмма плотности распределения значений')
plt.show()

sns.boxplot(x=data['comms_views'])
plt.xlabel('Значения')
plt.title('Коробчатая диаграмма распределения значений')
plt.show()

In [ ]:
Q1 = data['comms_views'].quantile(0.25)
Q3 = data['comms_views'].quantile(0.75)
IQR = Q3 - Q1
print(Q1, Q3, IQR)
# Определение границ для выбросов
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

In [ ]:
data_quantile = data[(data['comms_views'] >= lower_bound) & (data['comms_views'] <= upper_bound)]
data_quantile.info()

In [ ]:
sns.histplot(data_quantile['comms_views']*100, bins=5, kde=True)
plt.xlabel('Значения')
plt.ylabel('Частота')
plt.title('Гистограмма распределения значений с KDE')
plt.show()

sns.kdeplot(data_quantile['comms_views']*100, shade=True)
plt.xlabel('Значения')
plt.title('Диаграмма плотности распределения значений')
plt.show()

sns.boxplot(x=data_quantile['comms_views']*100)
plt.xlabel('Значения')
plt.title('Коробчатая диаграмма распределения значений')
plt.show()

In [ ]:
sns.histplot(data['forwards_views'], bins=5, kde=True)
plt.xlabel('Значения')
plt.ylabel('Частота')
plt.title('Гистограмма распределения значений с KDE')
plt.show()

sns.kdeplot(data['forwards_views'], shade=True)
plt.xlabel('Значения')
plt.title('Диаграмма плотности распределения значений')
plt.show()

sns.boxplot(x=data['forwards_views'])
plt.xlabel('Значения')
plt.title('Коробчатая диаграмма распределения значений')
plt.show()

In [ ]:
Q1 = data['forwards_views'].quantile(0.25)
Q3 = data['forwards_views'].quantile(0.75)
IQR = Q3 - Q1
print(Q1, Q3, IQR)
# Определение границ для выбросов
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

data_quantile = data[(data['forwards_views'] >= lower_bound) & (data['forwards_views'] <= upper_bound)]
data_quantile.info()

In [ ]:
sns.histplot(data_quantile['forwards_views']*100, bins=5, kde=True)
plt.xlabel('Значения')
plt.ylabel('Частота')
plt.title('Гистограмма распределения значений с KDE')
plt.show()

sns.kdeplot(data_quantile['forwards_views']*100, shade=True)
plt.xlabel('Значения')
plt.title('Диаграмма плотности распределения значений')
plt.show()

sns.boxplot(x=data_quantile['forwards_views']*100)
plt.xlabel('Значения')
plt.title('Коробчатая диаграмма распределения значений')
plt.show()

In [ ]:
sns.histplot(data['reactions_views'], bins=5, kde=True)
plt.xlabel('Значения')
plt.ylabel('Частота')
plt.title('Гистограмма распределения значений с KDE')
plt.show()

sns.kdeplot(data['reactions_views'], shade=True)
plt.xlabel('Значения')
plt.title('Диаграмма плотности распределения значений')
plt.show()

sns.boxplot(x=data['reactions_views'])
plt.xlabel('Значения')
plt.title('Коробчатая диаграмма распределения значений')
plt.show()

In [ ]:
Q1 = data['reactions_views'].quantile(0.25)
Q3 = data['reactions_views'].quantile(0.75)
IQR = Q3 - Q1
print(Q1, Q3, IQR)
# Определение границ для выбросов
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

data_quantile = data[(data['reactions_views'] >= lower_bound) & (data['reactions_views'] <= upper_bound)]
data_quantile.info()

In [ ]:
sns.histplot(data_quantile['reactions_views']*100, bins=5, kde=True)
plt.xlabel('Значения')
plt.ylabel('Частота')
plt.title('Гистограмма распределения значений с KDE')
plt.show()

sns.kdeplot(data_quantile['reactions_views']*100, shade=True)
plt.xlabel('Значения')
plt.title('Диаграмма плотности распределения значений')
plt.show()

sns.boxplot(x=data_quantile['reactions_views']*100)
plt.xlabel('Значения')
plt.title('Коробчатая диаграмма распределения значений')
plt.show()

# Создание метрики


In [ ]:
# Попробовал сделать таргет среднегармонческим,но слишком плохое выходит распределнеие -
# сильно прекошенное
data['harmonic'] = data.apply(lambda row: hmean([row['comms_views'], row['forwards_views'], row['reactions_views']]), axis=1)
data.head()

In [ ]:
# Поэтому пока что будем брать среднее конверсии (пока работаю над учетом смайлов)
data['mean'] = data[['comms_views', 'forwards_views', 'reactions_views']].mean(axis=1)
data.head()

## Сохранение c основными метриками


In [ ]:
data.to_csv('data_with_harmonic_mean.csv', index=False)

# Манипуляции с таргетом

In [ ]:
data = pd.read_csv('data_with_harmonic_mean.csv', sep = ',', parse_dates=['date'], encoding = 'utf-8')
data.head()

In [ ]:
"""

Здесь я тестирую возможности преобразования таргета и
просматриваю его реализацию.
В итоге пришел к тому, что использую среднее значение конверсии,логарифмирую
его и потом делаю преобразование Бокса-Кокса.
Получил плюс минус равномернораспределенный таргет и преобразовал его в классы,
перейдя от дискретного распределения с регрессией к классификации трех классов:
Низкий интерес, средний и высокий.
На практике видно, что у нас будет дисбаланс классов,
потому что обычных постов подавляющее большинство, однако сам выбор разделения
классов мне показался логичным - в жизни и правда мне так кажется и при ручном
просмотре по субьективному мнению очень похожу на правду. В этом моменте может
надо будет еще подумать что делать, но пока так.

"""

In [ ]:
data['harmonic'] = data['harmonic']*100
data['mean'] = data['mean']*100

data.drop(columns=['reactions', 'comments', 'views', 'reactions_views', 'comms_views', 'forwards_views', 'forwards'], inplace=True)
data.head()


In [ ]:
sns.histplot(data['mean'], bins=10, kde=True)
plt.xlabel('Значения')
plt.ylabel('Частота')
plt.title('Гистограмма распределения значений с KDE')
plt.show()

In [ ]:
Q1 = data['mean'].quantile(0.25)
Q3 = data['mean'].quantile(0.75)
IQR = Q3 - Q1
print(Q1, Q3, IQR)
# Определение границ для выбросов
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

data_quantile = data[(data['mean'] >= lower_bound) & (data['mean'] <= upper_bound)]
data_quantile.info()

In [ ]:
sns.histplot(data_quantile['mean'], bins=10, kde=True)
plt.xlabel('Значения')
plt.ylabel('Частота')
plt.title('Гистограмма распределения значений с KDE')
plt.show()


In [ ]:
sns.histplot(data['harmonic'], bins=10, kde=True)
plt.xlabel('Значения')
plt.ylabel('Частота')
plt.title('Гистограмма распределения значений с KDE')
plt.show()


In [ ]:
Q1 = data['harmonic'].quantile(0.25)
Q3 = data['harmonic'].quantile(0.75)
IQR = Q3 - Q1
print(Q1, Q3, IQR)
# Определение границ для выбросов
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

data_quantile = data[(data['harmonic'] >= lower_bound) & (data['harmonic'] <= upper_bound)]
data_quantile.info()

In [ ]:
sns.histplot(data_quantile['harmonic'], bins=10, kde=True)
plt.xlabel('Значения')
plt.ylabel('Частота')
plt.title('Гистограмма распределения значений с KDE')
plt.show()


In [ ]:
data.describe()

In [ ]:
def add_epsilon_to_zero(series, epsilon=1e-10):
    """
    Добавляет малое значение epsilon к нулевым элементам дял логарифмирования.
    """
    return series.apply(lambda x: x + epsilon if x == 0 else x)


In [ ]:
data['target1'] = add_epsilon_to_zero(data['mean'])
data['target2'] = add_epsilon_to_zero(data['harmonic'])


In [ ]:
data['target1'] = np.log1p(data['target1'])
data['target2'] = np.log1p(data['target2'])

data.describe()

In [ ]:
sns.histplot(data['target1'], bins=10, kde=True)
plt.xlabel('Значения')
plt.ylabel('Частота')
plt.title('Гистограмма распределения значений с KDE')
plt.show()


In [ ]:
sns.histplot(data['target2'], bins=10, kde=True)
plt.xlabel('Значения')
plt.ylabel('Частота')
plt.title('Гистограмма распределения значений с KDE')
plt.show()

In [ ]:
data['target11'], _ = boxcox(data['target1'])
data['target22'], _ = boxcox(data['target2'])
data.describe()

In [ ]:
sns.histplot(data['target11'], bins=10, kde=True)
plt.xlabel('Значения')
plt.ylabel('Частота')
plt.title('Гистограмма распределения значений с KDE')
plt.show()

In [ ]:
sns.histplot(data['target22'], bins=10, kde=True)
plt.xlabel('Значения')
plt.ylabel('Частота')
plt.title('Гистограмма распределения значений с KDE')
plt.show()

Окончательно перешли к таргету по среднему значению

In [ ]:
# Выбирал вручную
thresholds = data['target11'].quantile([0.15, 0.66]).values
thresholds

In [ ]:
def classify(value):
    if value <= thresholds[0]:
        return 'низкий'
    elif value <= thresholds[1]:
        return 'средний'
    else:
        return 'высокий'

data['class'] = data['target11'].apply(classify)
data.head()

In [ ]:
data.drop(columns=['subscribers', 'harmonic', 'mean', 'target1', 'target2', 'target11', 'target22'], inplace=True)
data.head()

In [ ]:
data[(data['name']=='BetBoom Esports Dota 2') & (data['class']=='низкий')].head()

In [ ]:
data[(data['name']=='BetBoom Esports Dota 2') & (data['class']=='высокий')].head()

## Сохранение с классами

In [ ]:
data.to_csv('data_with_class.csv', index=False)

In [ ]:
data = pd.read_csv('data_with_class.csv', sep = ',', parse_dates=['date'], encoding = 'utf-8')
data.head()

# Clean text



In [ ]:
class Cleaner:
    """
    Класс Cleaner предназначен для предварительной обработки текста
    перед его анализом. Он включает в себя функции для удаления HTML-тегов,
    URL, специальных символов, преобразования чисел и эмоджи в слова,
    удаления стоп-слов и знаков препинания, а также лемматизации.

    Methods:
        clean_text(self, input_text): Очищает входной текст от различных
        нежелательных элементов и приводит его к стандартному виду.

        emojis_words(self, text): Преобразует эмоджи в слова, используя их
         текстовое описание.

    Example:
        cleaner = Cleaner()
        raw_text = "Some raw text with HTML <html>...</html>,
                    URLs http://example.com, and emojis 😊."
        clean_text = cleaner.clean_text(raw_text)
        print(clean_text)
    """

    def __init__(self):
        """
        Инициализирует экземпляр класса Cleaner,
        загружая необходимые ресурсы для лемматизации и токенизации.
        """

        self.model_lemmatizer = pymorphy2.MorphAnalyzer()
        self.nlp = spacy.load("ru_core_news_sm")

    # Функция для очистки текста
    def clean_text(self, input_text):
        """
        Очищает входной текст от HTML-тегов, URL, эмоджи, специальных символов,
        преобразует числа в слова, удаляет стоп-слова и знаки препинания,
        и проводит лемматизацию.

        Params:
            input_text (str): Входной текст для очистки.

        Exit:
            str: Очищенный текст.
        """

        # HTML-теги: первый шаг - удалить из входного текста все HTML-теги
        clean_text = re.sub('<[^<]+?>', '', input_text)

        # URL и ссылки: далее - удаляем из текста все URL и ссылки
        clean_text = re.sub(r'http\S+', '', clean_text)

        # Эмоджи и эмотиконы: используем собственную функцию для преобразования эмоджи в текст
        # Важно понимать эмоциональную окраску обрабатываемого текста
        clean_text = self.emojis_words(clean_text)

        # Приводим все входные данные к нижнему регистру
        clean_text = clean_text.lower()

        # Убираем все пробелы
        # Так как все данные теперь представлены словами - удалим пробелы
        clean_text = re.sub('\s+', ' ', clean_text)

        # Убираем специальные символы: избавляемся от всего, что не является "словами"
        clean_text = re.sub('[^A-Za-zА-Яа-я0-9\s]', '', clean_text)

        # Записываем числа прописью: 100 превращается в "сто" (для компьютера)
        clean_text = self.replace_numbers_with_words(clean_text)

        # Стоп-слова: удаление стоп-слов - это стандартная практика очистки текстов
        # Лемматизируем каждое слово
        stop_words_ru = set(stopwords.words('russian'))
        stop_words_en = set(ENGLISH_STOP_WORDS)
        all_stop_words = stop_words_en.union(stop_words_ru)
        tokens = self.nlp(clean_text)
        tokens = [token.lemma_ for token in tokens if token not in all_stop_words]
        clean_text = ' '.join(tokens)

        # Знаки препинания: далее - удаляем из текста все знаки препинания
        clean_text = re.sub(r'[^\w\s]', '', clean_text)

        # clean_text = ' '.join([self.model_lemmatizer.parse(word)[0].normal_form for word in clean_text.split()])

        # И наконец - возвращаем очищенный текст
        return clean_text

    # Функция для преобразования чисел в слова
    def replace_numbers_with_words(self, text):
        """
        Преобразует числа в тексте в их словесные эквиваленты.

        Params:
            text (str): Текст для преобразования чисел.

        Exit:
            str: Текст с преобразованными числами.
        """
        # Паттерн для нахождения чисел
        number_pattern = r'\b\d+\b'

        # Нахождение эквивалента числу в словесной форме с помощью модуля num2words
        def replace(match):
            number = int(match.group(0))

            return num2words(number, lang='ru')

        return re.sub(number_pattern, replace, text)

    # Функция для преобразования эмоджи в слова
    def emojis_words(self, text):
        """
        Преобразует эмоджи в тексте в их словесные описания.

        Params:
            text (str): Текст для преобразования эмоджи.

        Exit:
            str: Текст с преобразованными эмоджи.
        """
        # Модуль emoji: преобразование эмоджи в их словесные описания
        clean_text = emoji.demojize(text, delimiters=(" ", " "))

        # Редактирование текста путём замены ":" и" _", а так же - путём добавления пробела между отдельными словами
        clean_text = clean_text.replace(":", "").replace("_", " ")

        return clean_text

In [ ]:
cleaner = Cleaner()
data['ctext'] = data['text'].apply(cleaner.clean_text)

In [ ]:
data[['ctext','text']]

## Сохранение очищенного текста

In [ ]:
data.to_csv('data_ctext.csv', index=False)

# Text (topic modeling)


In [ ]:
data = pd.read_csv('data_ctext.csv', sep = ',', parse_dates=['date'], encoding = 'utf-8')
data.head()

In [ ]:
vectorizer_model = CountVectorizer()
topic_model = BERTopic(vectorizer_model=vectorizer_model, language="multilingual")

In [ ]:
data.info()

In [ ]:
data[data['ctext'].isna()]

In [ ]:
data = data.dropna(subset='ctext')
data.info()

In [ ]:
topics, probs = topic_model.fit_transform(data['ctext'])

In [ ]:
topic_model.visualize_topics()

In [ ]:
data['topic'] = topics
data.head()

In [ ]:
data[data['topic'] == -1]

## Сохранение после topic modeling

In [ ]:
data.to_csv('data_with_topics.csv', index=False)

# TF-IDF

In [ ]:
data = pd.read_csv('data_with_topics.csv', sep = ',', parse_dates=['date'], encoding = 'utf-8')
data.head()

In [ ]:
data = data[['emoji', 'ctext', 'topic', 'class']]
data.head()

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_features=500, dtype=np.float32)
tfidf_matrix = tfidf_vectorizer.fit_transform(data['ctext'])
tfidf_scores = tfidf_matrix.toarray()
pca = PCA(n_components=10, random_state=random_state)
pca_tfidf = pd.DataFrame(pca.fit_transform(tfidf_scores))

In [ ]:
data_tfidf_pca = pd.concat([data, pca_tfidf], axis=1)
data_tfidf_pca.head()

In [ ]:
data_for_classification = data_tfidf_pca[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 'topic', 'class']]
data_for_classification.head()

## Сохранение после tf-idf

In [ ]:
data_for_classification.to_csv('data_for_classification.csv', index=False)

# Classification


 Сравнил несколько методов, XBoost дал лучший результат, для улучшения модели будут проводиться эксперименты с предобработкой данных и вытаскиванием фичей из текста, а также улучшение самой модели.

In [ ]:
data = pd.read_csv('data_for_classification.csv', sep = ',', encoding = 'utf-8')
data.head()

In [ ]:
X = data.drop(['class'], axis=1)
y = data['class']

scaler = StandardScaler()
X = scaler.fit_transform(X)
le = LabelEncoder()
y = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=random_state
                                                    )


## Logistic Regression

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

## SVM

In [ ]:
model = SVC()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

## Decision Trees

In [ ]:
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

## Random Forest

In [ ]:
model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

## Gradient Boosting

In [ ]:
model = GradientBoostingClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

## XGBoost

In [ ]:
model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Emojis (Beta)

In [ ]:
good_emojis = set([
    '😀', '😃', '😄', '😁', '😆', '😊', '😇', '🙂', '🙃', '😍',
    '😘', '😗', '😙', '😚', '😋', '😜', '😝', '😛', '🤑',
    '🤗', '🤩', '😎', '🤓', '😺', '😸', '😹', '😻', '😼', '😽'
])

bad_emojis = set([
    '😞', '😠', '😡', '😢', '😭', '😖', '😣', '😫', '😩', '😤',
    '😮', '😱', '😨', '😰', '😥', '😓', '😧', '😵', '😷', '🤒',
    '🤕', '🤢', '🤮', '🤧', '😪', '😴', '😵', '😲', '🤯'
])

def classify_emojis(text):
    text_emojis = set(char for char in text if char in emoji.UNICODE_EMOJI['en'])
    good_count = len(text_emojis & good_emojis)
    bad_count = len(text_emojis & bad_emojis)
    if good_count > bad_count:
        return 'good'
    elif bad_count > good_count:
        return 'bad'
    else:
        return 'neutral'

In [ ]:
def emoji_(text):
    text_emojis = emoji.distinct_emoji_list(text)
    return text_emojis

In [ ]:
emojis_in_txt = data['text'].apply(emoji_)
emojis_in_txt


In [ ]:
all_emojis_in_txt = sum(emojis_in_txt.tolist(), [])
len(all_emojis_in_txt)


In [ ]:
all_unic_emoji_in_txt = set(all_emojis_in_txt)
len(all_unic_emoji_in_txt)

In [ ]:
emojis = data['emoji'].apply(emoji_)
emojis


In [ ]:
all_emojis = sum(emojis.tolist(), [])
len(all_emojis)

In [ ]:
all_unic_emoji = set(all_emojis)
len(all_unic_emoji)

In [ ]:
intersection_emojis = all_unic_emoji & all_unic_emoji_in_txt
len(intersection_emojis)

In [ ]:
all_unic_emoji_list = list(all_unic_emoji)
all_unic_emoji_list[:10]

In [ ]:
analyzer = SentimentIntensityAnalyzer()

def analyze_sentiment_emoji(emoji_char):
    sentiment_score = analyzer.polarity_scores(emoji.demojize(emoji_char))
    return sentiment_score

In [ ]:
analyze_sentiment_emoji('❤‍🔥')

In [ ]:
for i in all_unic_emoji_list:
    print(i, analyze_sentiment_emoji(i))

In [ ]:
all_unic_emoji_list_in_str = []
for i in all_unic_emoji_list:
    all_unic_emoji_list_in_str.append(emoji.demojize(i))
all_unic_emoji_list_in_str

In [ ]:
def analyze_sentiment(emoji_char):
    sentiment_score = analyzer.polarity_scores(emoji_char)
    return sentiment_score

In [ ]:
for i in all_unic_emoji_list_in_str:
    print(i, analyze_sentiment(i))

In [ ]:
!pip install afinn

In [ ]:
from afinn import Afinn
afinn = Afinn()

In [ ]:
def analyze_sentiment_afinn(emoji_char):
    sentiment_score = afinn.score(emoji.demojize(emoji_char))
    return sentiment_score

In [ ]:
all_unic_emoji_list_semantic = [
    -1,
    0,
    0,
    0,
    1, #5
    0,
    -1,
    0,
    0,
    1, #10
    0,
    1,
    -1,
    1,
    1, #15
    -1,
    -1,
    0,
    1,
    1, #20
    0,
    0,
    0,
    1,
    0, #25
    0,
    1,
    1,
    -1,
    1, #30
    -1,
    1,
    1,
    1,
    1, #35
    1,
    1,
    -1,
    0,
    0, #40
    -1,
    1,
    1,
    0,
    -1, #45
    0,
    1,
    0,
    1,
    0, #50
    1,
    0,
    1,
    0,
    -1, #55
    0,
    -1,
    -1,
    1,
    0, #60
    1,
    0,
    0,
    0,
    0, #65
    1,
    0,
    1,
    1,
    0, #70
    0,
    -1,
    0
]

In [ ]:
#РАЗМЕТИЛ ВРУЧНУЮ
reactions_semantic = dict(zip(all_unic_emoji_list, all_unic_emoji_list_semantic))
reactions_semantic

In [ ]:
k = 1
for i in all_unic_emoji_list:
    print(k, i, analyze_sentiment_afinn(i))
    k+=1

In [ ]:
data['emoji'][200:240]

In [ ]:
emoji.is_emoji('❤\u200d🔥')

# Summ (beta)

In [ ]:


# Инициализация пайплайна
pipe = pipeline('summarization', model='d0rj/ru-mbart-large-summ')





In [ ]:
def summarize_posts(posts, model, tokenizer, max_length=150, min_length=50):
    individual_summaries = []
    for post in posts:
        summary = model(post, max_length=max_length, min_length=min_length, do_sample=False)
        individual_summaries.append(summary[0]['summary_text'])
    combined_text = ' '.join(individual_summaries)
    final_summary = model(combined_text, max_length=max_length, min_length=min_length, do_sample=False)
    return final_summary[0]['summary_text']


In [ ]:
# Пример постов
posts = [
    "Сегодня в мире произошло множество событий. В Японии произошло землетрясение магнитудой 6.5, не вызвавшее серьёзных разрушений, но повлиявшее на транспортное сообщение. В США президент подписал новый законопроект о здравоохранении, который направлен на снижение цен на лекарства. В России начался чемпионат по хоккею с участием 16 команд.",
    "Компания XYZ опубликовала квартальный отчёт, показывающий увеличение прибыли на 20% по сравнению с прошлым годом. Основные драйверы роста включают успешные маркетинговые кампании и увеличение продаж в онлайн-сегменте. В следующем квартале компания планирует расширить своё присутствие в Европе.",
    "Google анонсировала новую версию своей операционной системы Android, которая включает множество новых функций, таких как улучшенная безопасность, интеграция с IoT-устройствами и поддержка 5G. Ожидается, что обновление станет доступным в конце года.",
    "Психологи утверждают, что для достижения успеха важно придерживаться режима дня и ставить реалистичные цели. Начните с маленьких шагов, планируйте свой день и записывайте достижения. Постепенно вы сможете повысить свою продуктивность и достичь больших целей."
]

# Суммаризация постов
final_summary = summarize_posts(posts, pipe, None)  # None для tokenizer так как он уже встроен в модель
print("Final Summary:")
print(final_summary)